# 단어 벡터화 하기

In [1]:
# !pip install soynlp
!pip show soynlp

Name: soynlp
Version: 0.0.45
Summary: Unsupervised Korean Natural Language Processing Toolkits
Home-page: https://github.com/lovit/soynlp
Author: Lovit
Author-email: soy.lovit@gmail.com
License: UNKNOWN
Location: /Users/corazzon/codes/jupyter/lib/python3.6/site-packages
Requires: numpy, psutil
Required-by: 


In [2]:
# !pip install -U scikit-learn
!pip show scikit-learn

Name: scikit-learn
Version: 0.20.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /Users/corazzon/codes/jupyter/lib/python3.6/site-packages
Requires: numpy, scipy
Required-by: soykeyword, sklearn, predictive-imputer, mglearn, lightgbm


# 데이터 불러오기

In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
df = pd.read_csv('data/petition.csv', parse_dates=['start', 'end'])

In [5]:
# 관심사별로 텍스트 데이터를 학습시키기 위한 목적도 있지만 전체 텍스트를 돌렸을 때 빠르게 결과를 얻기 위해서 샘플링 하려는 의도도 있습니다.
p = r'.*(P2P|은행|금융|주식|증권|공매도).*'
finance = df[df['title'].str.match(p) |
           df['content'].str.match(p, flags=re.MULTILINE)]
finance.shape

(19866, 8)

In [6]:
finance.head()

,article_id,start,end,answered,votes,category,title,content
7,28,2017-08-19,2017-08-26,0,2137,경제민주화,소액주주를 보호해주십시오,** 존경하옵는 문재인대통령님께\n저는 중국원양자원이라는 KOSPI상장사의 소액 ...
11,32,2017-08-19,2017-09-03,0,30,경제민주화,소액주주 보호를 위해 조사요청 드립니다.,"존경하는 대통령님께,\n코스피 주식종목 중국 원양자원은 현재 계속되는 장** 대표이..."
54,77,2017-08-20,2017-09-04,0,53,교통/건축/국토,"안양시청, 안양지청, 재개발조합 적폐가 심해 수사가 안되고 있습니다. 안양시 지장물...","※지장물관련 국토부의 해석 : 기존 건축물의 철거 공사에 관한 사항의 범위는 전기,..."
68,91,2017-08-20,2017-09-19,0,165,경제민주화,증권 자본시장개정안(상장폐지법 개정안),존경하는 문제인 대통령님께\n먼저 어렵고 힘든 국정운영을 이끄시느라 노고가 많으십니...
100,124,2017-08-20,2017-08-27,0,1,기타,포스코 관련,안녕하십니까?\n최근 뉴스에 주목을 못 받는 뉴스가 있습니다. 아시아경제tv의 포스...


In [7]:
finance.tail()

,article_id,start,end,answered,votes,category,title,content
377603,491848,2019-01-09,2019-02-08,0,5,기타,은행 업무시간 연장 및 추가 인원 고용,은행 업무 시간을 오후 9시까지 연장해 직장인들도 편리하게 이용할 수 있도록 제안합...
377634,491885,2019-01-09,2019-02-08,0,5,정치개혁,국민연금폐지 하지않으면삶이피폐해진다,35년이 지나면 월급의 30%정도를. 국민연금보험료로 내야합니다.\n계산해보세요\n...
377662,491918,2019-01-09,2019-02-08,0,23,경제민주화,문정부 공매도폐지하라,개미들 돈털어가는 역적 문재인 공매도세력\n모두들 한국 증시 살리기위해 공매도 폐지...
377674,491935,2019-01-09,2019-02-08,0,11,육아/교육,베트남 유학생들에게 10개월째 학비를 돌려주지 않는 서울 ‘**대학’,안녕하세요. 여기는 베트남의 수도 하노이입니다. 최근의 이곳 상황은 베트남 축구대표...
377722,491998,2019-01-09,2019-02-08,0,1,기타,진실은 뉴스가 아니라 마음과 행동입니다,부동산\n새해 복 많이 받으세요\n정말 고생 하시고 수고 많으세요\n전 부동산 개발...


# 일부 텍스트 데이터 보기
* 어떤 데이터에 대한 전처리가 필요할지 봅니다.

In [8]:
# 샘플로 보고 싶은 인덱스의 번호를 넣어주세요.
sample_index = 192190

In [9]:
sample_title = finance['title'][sample_index]
sample_title

'2009년도 탄생 적폐제도(?) 신속(조기)집행 폐지를 간절히 청원합니다.'

In [10]:
sample_content = finance['content'][7]
sample_content[:1000]

"**  존경하옵는 문재인대통령님께\\n저는 중국원양자원이라는 KOSPI상장사의 소액 주주입니다. 중국원양자원(이하 중원)은 이번 재회계감사에서 의견거절이라는 감사인 판단이 내려져 꿈에도 생각못한 상장폐지를 목전에 두고 있습니다.\\n상장폐지가되면 국내에 상장하여 천문학적인 금액을 취득하여 회사의 규모를 키우고 개인자산을 불린 후 자신의 지분도  다 팔아서 이득을 챙긴 중국인 장화리 사장은 정리매매때 휴지조각의 비용으로 자신의 지분을 다시 사들이고 대주주로서의 지위를 다시 획득하여 온전히 자신의 기업으로 만들수 있습니다.\\n결국 한국 국민들의 피땀어린 투자금으로 중국인 사장의 부를 채워주지만 이에 대해 한국의 금융당국 및 상장과 운영의 주체인 거래소는 누구도 이에대한 책임없이 이를 그냥 덮어버리고만 말것입니다.\\n이로인해 약 2만 5천의 국내소액주주는 피눈물을 흘리며 고통속에 살게 될 것입니다.\\n주식투자는 개인의 판단하에 이루어지는 것이나 투자에 대한 개인의 책임을 묻기전에 그 과정은 규정과 상식에 맞게 공정하고 정의로워야 할 것입니다. 이는 존경하는 문대통령님의 말씀이라 가슴깊이 새기고 있었으나 현재 우리나라의 금융 시스템, 특히 거래소와 회계감사는 그렇지 못한 것 같습니다.\\n이렇게 생각하게된 이유는 이번 회계감사의 주된 의견거절의 이유 때문입니다\\n이번 신한회계법인의 추측적이고 터무니없는 의견거절로 전재산이 한순간에 물거품으로 사라질 위기에 처한 본인을 비롯한 많은 사람들이 죽음이라는 극단적인 선택앞에 내몰려져 억울함과 원통함에 피눈물을 흘리고 있습니다. .\\n기업의 사업성이나 계속 존속여부를 점검하는 감사자인 회계법인은 오직 밝혀진 사실과 그에 따른 증거만을 근거로 공정하게 기업의 존속여부에 대한 의견을 내야하고 유추할 경우에도 당연히 관련 법이나 판례에 근거하여야 합니다.\\n그러나 신한회계법인은 법또는 제도권으로 확정되지 않은 추정사항을 의견거절의'결정적 이유'로 명시하였는바 이는 감사자의 '의견거절'에 따른 참혹한 결과로 볼때 2500

In [11]:
sample_content = finance['content'][sample_index]
sample_content[:1000]

'2009년도 탄생 적폐제도(?) 신속(조기)집행 폐지를 간절히 청원합니다.\\n국민의 일원으로써, 일선지자체의 공무원으로써,\\n2009년도부터 실시되어 현재까지 시행중인 적폐 제도, 신속(조기,균형) 집행의 폐지를 청원합니다.\\n2008년 리먼사태로 세계 금융위기가 발생하였다는 이유로 2009년도에 시작된 조기집행...\\n(지극히 제 개인적인 생각입니다다만. 지나고 나서 돌아보니, 솔직히 그것 때문에 시작했다고 생각이 들지 않습니다.)\\n어찌되었든, 현재까지 10년동안 그 제도같지 않은 제도가 이름만 달리하면서(조기→균형→신속) 시행되고 있는 것으로 보아,\\n우리 국가는 매년을 급박한 경제위기속에서 신속(조기)집행으로 버티어 내고 있는 중인 것으로 보입니다.\\n또한, 해당 제도 시행의 근거로 제시되는 「상반기 어려운 고용상황 및 거시경제 불안요인 등에 대응」에 따르면, 우리 국가는 금방이라도 부도가 날 지경인 것 같습니다.\\n현재 우리 대한민국은 정말 그러한 상황인가 되묻고 싶습니다.\\n지난 10년동안 각종 폐해와 문제점이 지적되고 있음에도, 도대체 어떤 이유로 현재까지 시행이 되고 있는지 이유를 찾을 수가 없습니다.\\n각종 언론매체 등을 통해 제기된 부작용을 나열하면,\\n“단기간 과다한 공사발주로 부실 설계 우려”,\\n“지자체 이자 수입 감소와 재정 압박 초래”,\\n“발주 후 설계변경이나 중복 집행 등의 예산낭비”,\\n“공사가 단기간에 집중되면서 공사현장에서 인건비나 자재비가 급등”,\\n“획일적 조기집행의 추진은 지방재정 안정성을 저해”,\\n“중앙정부의 예산조기집행은 지방자치와 지방재정 건전성 문제에 위배”\\n등의 문제점이 전문가, 지자체장, 지방의원, 공무원 등에 의해 지속적으로 제기되어 왔습니다.\\n또한 행정안전부에서도 이미 그 폐해, 문제점을 인지하고 있습니다.\\n-2018년도 행정안전부 지방재정 신속집행 추진계획 3페이지\\n○자치단체, 지방의회, 공무원 단체 등에서 신속집행에 대한 문제점* 등을 들어 지속적으로 개선

# 텍스트 데이터 전처리 
기계가 텍스트를 이해할 수 있도록 텍스트를 정제해 줍니다.

신호와 소음을 구분합니다. 아웃라이어데이터로 인한 오버피팅을 방지합니다.


## 텍스트 데이터 전처리 이해하기

(출처 : [트위터 한국어 형태소 분석기](https://github.com/twitter/twitter-korean-text))

**정규화 normalization (입니닼ㅋㅋ -> 입니다 ㅋㅋ, 샤릉해 -> 사랑해)**

* 한국어를 처리하는 예시입니닼ㅋㅋㅋㅋㅋ -> 한국어를 처리하는 예시입니다 ㅋㅋ

**토큰화 tokenization**

* 한국어를 처리하는 예시입니다 ㅋㅋ -> 한국어Noun, 를Josa, 처리Noun, 하는Verb, 예시Noun, 입Adjective, 니다Eomi ㅋㅋKoreanParticle

**어근화 stemming (입니다 -> 이다)**

* 한국어를 처리하는 예시입니다 ㅋㅋ -> 한국어Noun, 를Josa, 처리Noun, 하다Verb, 예시Noun, 이다Adjective, ㅋㅋKoreanParticle


**어구 추출 phrase extraction** 

* 한국어를 처리하는 예시입니다 ㅋㅋ -> 한국어, 처리, 예시, 처리하는 예시

Introductory Presentation: [Google Slides](https://docs.google.com/presentation/d/10CZj8ry03oCk_Jqw879HFELzOLjJZ0EOi4KJbtRSIeU/)

# 불용어 Stopwords

* word2vec으로 금융관련 키워드로 가장 빈번하게 등장하는 단어를 추출해 보니 다음과 같은 단어들이 나왔습니다다.
* 한국어 불용어는 정리된 라이브러리는 없는 편입니다. 상황에 따라 필요한 단어가 있을 수도 있고 불용어가 문맥에 따라 의미를 필요로 할때도 있기 때문입니다. 

['수', '있는', '있습니다', '그', '년', '합니다', '하는', '및', '제', '할', '하고', '더', '대한', '한', '그리고', '월', '저는', '없는', '것입니다', '등', '일', '많은', '이런', '것은', '왜', '같은', '없습니다', '위해', '한다']

In [12]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    # 공백이 여러개 있는 것을 하나로 줄여준다.
    text = re.sub(' +', ' ', text)
    return text

In [13]:
def remove_stopwords(text):
    stops = ['수', '있는', '있습니다', '그', '년도', '에', '합니다', 
             '하는', '및', '제', '할', '하고', '더', '대한', '한', 
             '그리고', '월', '저는', '없는', '것입니다', '등', '일', 
             '많은', '이런', '것은', '왜', '같은', 
             '없습니다', '위해', '한다']
    # Stopwords 불용어 제거
    meaningful_words = [w for w in text if not w in stops]
    return ''.join(meaningful_words)

In [14]:
# 샘플을 하나 가져와 전처리를 적용해 봅니다.
sample_content = preprocessing(sample_content)
sample_content[:1000]

' 년도 탄생 적폐제도 신속 조기 집행 폐지를 간절히 청원합니다 국민의 일원으로써 일선지자체의 공무원으로써 년도부터 실시되어 현재까지 시행중인 적폐 제도 신속 조기 균형 집행의 폐지를 청원합니다 년 리먼사태로 세계 금융위기가 발생하였다는 이유로 년도에 시작된 조기집행 지극히 제 개인적인 생각입니다다만 지나고 나서 돌아보니 솔직히 그것 때문에 시작했다고 생각이 들지 않습니다 어찌되었든 현재까지 년동안 그 제도같지 않은 제도가 이름만 달리하면서 조기 균형 신속 시행되고 있는 것으로 보아 우리 국가는 매년을 급박한 경제위기속에서 신속 조기 집행으로 버티어 내고 있는 중인 것으로 보입니다 또한 해당 제도 시행의 근거로 제시되는 상반기 어려운 고용상황 및 거시경제 불안요인 등에 대응 에 따르면 우리 국가는 금방이라도 부도가 날 지경인 것 같습니다 현재 우리 대한민국은 정말 그러한 상황인가 되묻고 싶습니다 지난 년동안 각종 폐해와 문제점이 지적되고 있음에도 도대체 어떤 이유로 현재까지 시행이 되고 있는지 이유를 찾을 수가 없습니다 각종 언론매체 등을 통해 제기된 부작용을 나열하면 단기간 과다한 공사발주로 부실 설계 우려 지자체 이자 수입 감소와 재정 압박 초래 발주 후 설계변경이나 중복 집행 등의 예산낭비 공사가 단기간에 집중되면서 공사현장에서 인건비나 자재비가 급등 획일적 조기집행의 추진은 지방재정 안정성을 저해 중앙정부의 예산조기집행은 지방자치와 지방재정 건전성 문제에 위배 등의 문제점이 전문가 지자체장 지방의원 공무원 등에 의해 지속적으로 제기되어 왔습니다 또한 행정안전부에서도 이미 그 폐해 문제점을 인지하고 있습니다 년도 행정안전부 지방재정 신속집행 추진계획 페이지 자치단체 지방의회 공무원 단체 등에서 신속집행에 대한 문제점 등을 들어 지속적으로 개선 요구 이자수입 대폭 감소 상반기 공사집중으로 부실시공 우려 공사자재 품귀현상 발생 지역경제 활성화 파급효과 미미 무리한 실적경쟁으로 막대한 행정력 낭비 제도의 성과 분석과 평가 없이 매년 반복 시행 등 국회 년 국정

In [15]:
# 샘플텍스트를 가져와 불용어를 제거해 봅니다. 
stop_content = remove_stopwords(sample_content)
stop_content[:1000]

' 년도 탄생 적폐도 신속 조기 집행 폐지를 간절히 청원합니다 국민의 원으로써 선지자체의 공무원으로써 년도부터 실시되어 현재까지 시행중인 적폐 도 신속 조기 균형 집행의 폐지를 청원합니다 년 리먼사태로 세계 금융위기가 발생하였다는 이유로 년도 시작된 조기집행 지극히  개인적인 생각입니다다만 지나고 나서 돌아보니 솔직히 것 때문 시작했다고 생각이 들지 않습니다 어찌되었든 현재까지 년동안  도같지 않은 도가 이름만 달리하면서 조기 균형 신속 시행되고 있는 것으로 보아 우리 국가는 매년을 급박 경위기속서 신속 조기 집행으로 버티어 내고 있는 중인 것으로 보입니다 또 해당 도 시행의 근거로 시되는 상반기 어려운 고용상황  거시경 불안요인  대응  따르면 우리 국가는 금방이라도 부도가 날 지경인 것 같습니다 현재 우리 대민국은 정말 러 상황인가 되묻고 싶습니다 지난 년동안 각종 폐해와 문점이 지적되고 있음도 도대체 어떤 이유로 현재까지 시행이 되고 있는지 이유를 찾을 가 없습니다 각종 언론매체 을 통해 기된 부작용을 나열하면 단기간 과다 공사발주로 부실 설계 우려 지자체 이자 입 감소와 재정 압박 초래 발주 후 설계변경이나 중복 집행 의 예산낭비 공사가 단기간 집중되면서 공사현장서 인건비나 자재비가 급 획적 조기집행의 추진은 지방재정 안정성을 저해 중앙정부의 예산조기집행은 지방자치와 지방재정 건전성 문 위배 의 문점이 전문가 지자체장 지방의원 공무원  의해 지속적으로 기되어 왔습니다 또 행정안전부서도 이미  폐해 문점을 인지하고 있습니다 년도 행정안전부 지방재정 신속집행 추진계획 페이지 자치단체 지방의회 공무원 단체 서 신속집행 대 문점 을 들어 지속적으로 개선 요구 이자입 대폭 감소 상반기 공사집중으로 부실시공 우려 공사자재 품귀현상 발생 지역경 활성화 파급효과 미미 무리 실적경쟁으로 막대 행정력 낭비 도의 성과 분석과 평가 없이 매년 반복 시행  국회 년 국정감사 년 국정감사 년 예결위 부별심사 시도 도개선 필요성 을 지적 선 지자체서 근무하는 공무원으로써 위 문점

In [16]:
# %time을 찍어주면 해당 코드를 실행할 때 걸리는 시간을 출력해 줍니다
%time finance['content_preprocessing'] = finance['content'].apply(preprocessing)
sentences = finance['content_preprocessing']
sentences.shape

CPU times: user 5 s, sys: 184 ms, total: 5.18 s
Wall time: 5.8 s


/Users/corazzon/codes/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(19866,)

In [17]:
sentences.head()

7       존경하옵는 문재인대통령님께 저는 중국원양자원이라는 KOSPI상장사의 소액 주주입니...
11     존경하는 대통령님께 코스피 주식종목 중국 원양자원은 현재 계속되는 장 대표이사의 기...
54      지장물관련 국토부의 해석 기존 건축물의 철거 공사에 관한 사항의 범위는 전기 가스...
68     존경하는 문제인 대통령님께 먼저 어렵고 힘든 국정운영을 이끄시느라 노고가 많으십니다...
100    안녕하십니까 최근 뉴스에 주목을 못 받는 뉴스가 있습니다 아시아경제tv의 포스코 비...
Name: content_preprocessing, dtype: object

# BOW(bag of words)
* 가장 간단하지만 효과적이라 널리쓰이는 방법
* 장, 문단, 문장, 서식과 같은 입력 텍스트의 구조를 제외하고 각 단어가 이 말뭉치에 얼마나 많이 나타나는지만 헤아립니다.
* 구조와 상관없이 단어의 출현횟수만 세기 때문에 텍스트를 담는 가방(bag)으로 생각할 수 있습니다.
* BOW는 단어의 순서가 완전히 무시 된다는 단점이 있다. 예를 들어 의미가 완전히 반대인 두 문장이 있다고 합니다.
    - `it's bad, not good at all.` 
    - `it's good, not bad at all.` 
* 위 두 문장은 의미가 전혀 반대지만 완전히 동일하게 반환됩니다.
* 이를 보완하기 위해 n-gram을 사용하는 데 BOW는 하나의 토큰을 사용하지만 n-gram은 n개의 토큰을 사용할 수 있도록 합니다.
* min_df는 문서에 특정 단어가 최소 몇 번 이상 문서에 등장하는 단어를 가방에 담겠다는 의미입니다.

* [Bag-of-words model - Wikipedia](https://en.wikipedia.org/wiki/Bag-of-words_model)

# 사이킷런의 CountVectorizer를 통해 벡터화
* 미리 전처리 해둔 텍스트 데이터로 벡터화 합니다.
* 모두 소문자로 변환시키기 때문에 영어의 경우 good, Good, gOod이 모두 같은 특성이 됩니다.
* 의미없는 특성을 많이 생성하기 때문에 적어도 두 번이상 문서에 나타난 토큰만을 사용한다.

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있습니다.
                             tokenizer = None, # 토크나이저를 따로 지정해 줄 수도 있습니다.
                             preprocessor = None, # 전처리 도구
                             stop_words = None, # 불용어 nltk등의 도구를 사용할 수도 있습니다.
                             min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3), # BOW의 단위를 1~3개로 지정합니다.
                             max_features = 1000 # 만들 피처의 수, 단어의 수가 된다.
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=2,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [19]:
%%time 
feature_vector = vectorizer.fit_transform(finance['content_preprocessing'])
feature_vector.shape

CPU times: user 1min 23s, sys: 5.43 s, total: 1min 28s
Wall time: 1min 35s


In [20]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

1000


['articleview',
 'articleview html',
 'articleview html idxno',
 'co',
 'co kr',
 'co kr news',
 'com',
 'html',
 'html idxno',
 'http']

In [21]:
# 벡터화 된 피처를 확인해 봄
# Bag of words 에 몇 개의 단어가 들어있는지 확인
dist = np.sum(feature_vector, axis=0)
    
# for tag, count in zip(vocab, dist):
#     print(count, tag)
    
pd.DataFrame(dist, columns=vocab)

,articleview,articleview html,articleview html idxno,co,co kr,co kr news,com,html,html idxno,http,...,회사는,회사를,회사에,회사의,회장,훨씬,희망을,힘든,힘들게,힘을
0,911,842,840,1343,1309,826,1590,1218,840,2323,...,533,600,546,779,453,534,583,745,722,475


In [22]:
# 각 row에서 전체 단어가방에 있는 어휘에서 등장하는 단어에 대한 one-hot-vector
pd.DataFrame(feature_vector[:10].toarray(), columns=vocab).head()

,articleview,articleview html,articleview html idxno,co,co kr,co kr news,com,html,html idxno,http,...,회사는,회사를,회사에,회사의,회장,훨씬,희망을,힘든,힘들게,힘을
0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,7,0,1,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0,0,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## TF-IDF
TF(단어 빈도, term frequency)는 특정한 단어가 문서 내에 얼마나 자주 등장하는지를 나타내는 값으로, 이 값이 높을수록 문서에서 중요하다고 생각할 수 있다. 하지만 단어 자체가 문서군 내에서 자주 사용되는 경우, 이것은 그 단어가 흔하게 등장한다는 것을 의미한다. 이것을 DF(문서 빈도, document frequency)라고 하며, 이 값의 역수를 IDF(역문서 빈도, inverse document frequency)라고 한다. TF-IDF는 TF와 IDF를 곱한 값이다.

IDF 값은 문서군의 성격에 따라 결정된다. 예를 들어 '원자'라는 낱말은 일반적인 문서들 사이에서는 잘 나오지 않기 때문에 IDF 값이 높아지고 문서의 핵심어가 될 수 있지만, 원자에 대한 문서를 모아놓은 문서군의 경우 이 낱말은 상투어가 되어 각 문서들을 세분화하여 구분할 수 있는 다른 낱말들이 높은 가중치를 얻게 된다.

역문서 빈도(IDF)는 한 단어가 문서 집합 전체에서 얼마나 공통적으로 나타나는지를 나타내는 값이다. 전체 문서의 수를 해당 단어를 포함한 문서의 수로 나눈 뒤 로그를 취하여 얻을 수 있다.

* 출처 : [TF-IDF - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/TF-IDF)

\begin{equation*}
\text{tfidf}(w, d) = \text{tf} \times (\log\big(\frac{N + 1}{N_w + 1}\big) + 1)
\end{equation*}


* 싸이킷런 공식문서 : [4.2. Feature extraction — scikit-learn 0.19.1 documentation](http://scikit-learn.org/stable/modules/feature_extraction.html)

### TfidfTransformer()
* norm='l2' 각 문서의 피처 벡터를 어떻게 벡터 정규화 할지 정한다. 
    - L2 : 벡터의 각 원소의 제곱의 합이 1이 되도록 만드는 것이고 기본 값
    - L1 : 벡터의 각 원소의 절댓값의 합이 1이 되도록 크기를 조절
* smooth_idf=False
    - 피처를 만들 때 0으로 나오는 항목에 대해 작은 값을 더해서(스무딩을 해서) 피처를 만들지 아니면 그냥 생성할지를 결정
* sublinear_tf=False
* use_idf=True
    - TF-IDF를 사용해 피처를 만들 것인지 아니면 단어 빈도 자체를 사용할 것인지 여부

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [24]:
%%time 
feature_tfidf = transformer.fit_transform(feature_vector)
feature_tfidf.shape

CPU times: user 64.7 ms, sys: 10.3 ms, total: 74.9 ms
Wall time: 81.3 ms


In [25]:
feature_tfidf

<19866x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 793346 stored elements in Compressed Sparse Row format>

In [26]:
# 벡터화 된 피처를 확인해 봄
# Bag of words 에 몇 개의 단어가 들어있는지 확인
dist = np.sum(feature_tfidf, axis=0)
# dist

In [27]:
# 단어와 함께 볼 수 있도록    
for tag, count in zip(vocab, dist):
    print(count, tag)
    
pd.DataFrame(dist, columns=vocab)

[[ 28.25133835  24.48062374  24.46200973  48.6410175   45.37175652
   23.65362873  66.48125832  39.15007425  24.46200973  85.7148902
   49.55993244  15.8925277   50.87874021  33.46039348  28.6208557
   63.34165561  26.63436174  21.96322786  15.8925277   15.8925277
   15.8925277   32.00842977  24.47629288  66.05255763  24.48062374
   24.48062374  73.06249862  15.8925277   15.8925277   45.11823027
   58.43597241  48.88632238  45.73337966  51.52756551  42.00703648
   56.90786699 327.27824874  80.82873289  83.88015437 101.02670017
   70.43909223  59.16280143 167.158621    58.93574168 180.51646846
   84.273947    53.02112701  61.31384946 115.72632403  75.97648948
   64.53471847  99.50978056  69.96205528  68.29710503  74.36613553
   64.51163757  43.87750042 183.17474304 289.35398754 177.87279349
  101.16037039 137.356022    82.69247959  69.16164264  35.73502726
  102.16501737  66.62571621  84.97033447 167.96177972  63.71275771
  112.49312996  64.24099421  69.28209009 115.24128696  40.1425207

,articleview,articleview html,articleview html idxno,co,co kr,co kr news,com,html,html idxno,http,...,회사는,회사를,회사에,회사의,회장,훨씬,희망을,힘든,힘들게,힘을
0,28.251338,24.480624,24.46201,48.641017,45.371757,23.653629,66.481258,39.150074,24.46201,85.71489,...,53.923562,62.03072,60.257623,66.003475,33.30038,46.985882,68.017486,77.082978,82.62476,52.874017


In [28]:
# 각 row에서 전체 단어가방에 있는 어휘에서 등장하는 단어에 대한 one-hot-vector에 TF-IDF 가중치 반영
tfidf_df = pd.DataFrame(feature_tfidf[:10].toarray(), columns=vocab)
tfidf_df.head() 

,articleview,articleview html,articleview html idxno,co,co kr,co kr news,com,html,html idxno,http,...,회사는,회사를,회사에,회사의,회장,훨씬,희망을,힘든,힘들게,힘을
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.042665,0.000000,0.041555,0.273172,0.0,0.040804,0.039747,0.000000,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.181909,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.075922,0.072577,0.000000,0.069444,0.0,0.000000,0.000000,0.066433,0.068521,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [29]:
# 특정 단어의 가중치
tfidf_df['힘든']

0    0.000000
1    0.000000
2    0.000000
3    0.066433
4    0.000000
5    0.000000
6    0.000000
7    0.000000
8    0.000000
9    0.000000
Name: 힘든, dtype: float64

In [38]:
# 특정 row에서 가중치가 적용된 값을 본다.
tfidf_df.iloc[0][-50:]

한국의       0.040369
한다        0.000000
한다고       0.061589
한다는       0.000000
한다면       0.000000
한번        0.000000
할수        0.000000
함께        0.032310
합니까       0.000000
합니다       0.037015
항상        0.000000
해결        0.000000
해당        0.000000
해도        0.000000
해서        0.000000
해야        0.000000
해야 합니다    0.000000
해외        0.000000
해주세요      0.000000
했고        0.000000
했는데       0.000000
했다        0.000000
했던        0.040079
했습니다      0.000000
했지만       0.000000
행위를       0.000000
향후        0.000000
현실입니다     0.000000
현재        0.095100
현재의       0.000000
현행        0.000000
혜택을       0.000000
혹은        0.000000
혼자        0.000000
화폐        0.000000
확인        0.000000
활성화       0.000000
황우석       0.000000
회사        0.181455
회사가       0.076660
회사는       0.042665
회사를       0.000000
회사에       0.041555
회사의       0.273172
회장        0.000000
훨씬        0.040804
희망을       0.039747
힘든        0.000000
힘들게       0.000000
힘을        0.000000
Name: 0, dtype: float64